In [5]:
%reload_ext watermark
%watermark -a "Eduardo Teles Guimaraes" --iversions

Author: Eduardo Teles Guimaraes

seaborn   : 0.11.2
numpy     : 1.21.5
pandas    : 1.4.4
matplotlib: 3.5.2
sklearn   : 1.0.2
scipy     : 1.9.1



In [6]:
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [11]:
np.random.normal(0, 1, 3 * 1) * np.sqrt(2 / 3)

array([-0.26354543, -0.75137839,  0.51091234])

In [ ]:
class Dense:
    '''
    Classe que cria a arquitetura da rede neural:
    entradas
    feat_size :: tamanho de entradas iniciais da arquitetura 1, 2, 3, .., n
    out_size :: geralmente 1 saida, ou entao, multicamadas
    '''
    def __init__(self, feat_size, out_size):
        # Tamanho das dimensoes
        self.feat_size = feat_size
        self.out_size = out_size
        # np.random.normal(0, 1, 3 * 1) * np.sqrt(2 / 3)    array([ 0.40406558, -0.27438716,  0.58409914])
        # Criando um vetor de peso alinhado com a dimensao de entrada e saida
        self.weights = (np.random.normal(0, 1, feat_size * out_size) * np.sqrt(2 / feat_size)).reshape(feat_size, out_size)
        #Avoid negative output
        self.bias = np.random.rand(1, out_size) - 0.5
    
    def forward(self, input_data):
        self.output = np.dot(input_data, self.weights) + self.bias

        return self.output

    def backpropagation(self, output_der, lr):
        # erro = output - output_layer
        # Calcular o erro -> (w+1) = (w(n) * erro * lr)
        pass
    





    